In [ ]:
!pip install sentencepiece transformers==4.33 datasets sacremoses sacrebleu  -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 19.4 MB/s eta 0:00:00


In [ ]:
import gc
import random
import numpy as np
import torch
import pandas as pd
import sacrebleu
import sentencepiece as spm
from sentencepiece import sentencepiece_model_pb2 as sp_pb2_model
from transformers import NllbTokenizer
from transformers import Adafactor, get_scheduler
from transformers import AutoModelForSeq2SeqLM
from transformers import get_constant_schedule_with_warmup
from tqdm.auto import tqdm, trange
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

In [ ]:
class Translator:
  def __init__(self, learningRate=1e-3, batchSize: int=16):
    self.tokenizer = NllbTokenizer.from_pretrained('facebook/nllb-200-distilled-600M')
    self.__fix_tokenizer()
    self.model = AutoModelForSeq2SeqLM.from_pretrained('facebook/nllb-200-distilled-600M')
    self.model.resize_token_embeddings(len(self.tokenizer))
    self.learningRate = learningRate
    self.batchSize = batchSize
    self.losses = []
    self.LANGS = [('eng', 'eng_Latn'), ('ami', 'ami_Latn')]


  def __setup(self):
    self.optimizer = Adafactor(
        [p for p in self.model.parameters() if p.requires_grad],
        scale_parameter=False,
        relative_step=False,
        lr=self.learningRate,  # Starting with a higher learning rate
        clip_threshold=1.0,
        weight_decay=1e-3,
      )

    self.scheduler = get_constant_schedule_with_warmup(
        self.optimizer,
        num_warmup_steps=500
      )


  def __word_tokenize(text: str):
    return re.findall('(\w+|[^\w\s])', text)


  def __generate_splits(self, data: object, split: bool):
    data.rename(columns={'ami': 'Amis', 'eng': 'English'}, inplace=True)

    if split:
      self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(
          data['Amis'],
          data['English'],
          random_state=104,
          test_size=0.2,
          shuffle=True
        )
    else:
      self.x_train = data[data['split'] == 'train']['Amis']
      self.y_train = data[data['split'] == 'train']['English']
      self.x_test = data[data['split'] == 'test']['Amis']
      self.y_test = data[data['split'] == 'test']['English']

    self.y_train['eng_toks'] = self.y_train.English.apply(self.tokenizer.tokenize)
    self.x_train['ami_toks'] = self.x_train.Amis.apply(self.tokenizer.tokenize)
    self.y_train['eng_words'] = self.y_train.English.apply(self.word_tokenize)
    self.x_train['ami_words'] = self.x_train.Amis.apply(self.word_tokenize)


  def __fix_tokenizer(self, new_lang: str='ami_Latn'):
    old_len = len(self.tokenizer) - int(new_lang in self.tokenizer.added_tokens_encoder)
    self.tokenizer.lang_code_to_id[new_lang] = old_len-1
    self.tokenizer.id_to_lang_code[old_len-1] = new_lang
    # always move "mask" to the last position
    self.tokenizer.fairseq_tokens_to_ids["<mask>"] = len(self.tokenizer.sp_model) + len(self.tokenizer.lang_code_to_id) + self.tokenizer.fairseq_offset

    self.tokenizer.fairseq_tokens_to_ids.update(self.tokenizer.lang_code_to_id)
    self.tokenizer.fairseq_ids_to_tokens = {v: k for k, v in self.tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in self.tokenizer._additional_special_tokens:
        self.tokenizer._additional_special_tokens.append(new_lang)
    # clear the added token encoder; otherwise a new token may end up there by mistake
    self.tokenizer.added_tokens_encoder = {}
    self.tokenizer.added_tokens_decoder = {}


  def __cleanup(self):
    gc.collect()
    torch.cuda.empty_cache()


  def __get_batch_pairs(self):
    xx, yy = [], []
    for _ in range(self.batchSize):
        src = self.x_train.iloc[random.randint(0, len(self.x_train)-1)]
        tgt = self.y_train.iloc[random.randint(0, len(self.y_train)-1)]
        xx.append(src)
        yy.append(tgt)
    return xx, yy, 'ami_Latn', 'eng_Latn'


  def __tokenize_and_transition(self, lang):
    return self.tokenizer(
        lang,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=128
      ).to(self.model.device)


  def __train_sentencepiece(self, text_file: str, spm_prefix: str):
      spm.SentencePieceTrainer.train(
          input=text_file,
          model_prefix=spm_prefix,
          vocab_size=2**14,  # 16K
          character_coverage=1,
          num_threads=16,
          train_extremely_large_corpus=False,
          add_dummy_prefix=False,
          max_sentencepiece_length=128,
          max_sentence_length=4192*4,
          pad_id=0,
          eos_id=1,
          unk_id=2,
          bos_id=-1,
      )


  def __load_sentencepiece_model(self, model_file: str):
      sp_trained = spm.SentencePieceProcessor(model_file=model_file)
      added_spm = sp_pb2_model.ModelProto()
      added_spm.ParseFromString(sp_trained.serialized_model_proto())
      return added_spm


  def __update_existing_model_with_new_tokens(self, added_spm: sp_pb2_model.ModelProto):
      old_spm = sp_pb2_model.ModelProto()
      old_spm.ParseFromString(self.tokenizer.sp_model.serialized_model_proto())

      nllb_tokens_set = {p.piece for p in old_spm.pieces}
      prev_min_score = old_spm.pieces[-1].score

      for p in added_spm.pieces:
          piece = p.piece
          if piece not in nllb_tokens_set:
              new_p = sp_pb2_model.ModelProto().SentencePiece()
              new_p.piece = piece
              new_p.score = p.score + prev_min_score
              old_spm.pieces.append(new_p)

      return old_spm


  def __save_updated_model(self, updated_spm: sp_pb2_model.ModelProto, new_spm_name: str):
      with open(new_spm_name, 'wb') as f:
          f.write(updated_spm.SerializeToString())


  def __update_tokenizer_and_model(self, new_spm_name: str):
      tokenizer_old = self.tokenizer
      self.tokenizer = NllbTokenizer.from_pretrained('facebook/nllb-200-distilled-600M', vocab_file=new_spm_name)
      self.model.resize_token_embeddings(len(self.tokenizer))

      added_vocab = set(self.tokenizer.get_vocab()).difference(set(tokenizer_old.get_vocab()))

      for t in tqdm(added_vocab):
          tt = tokenizer_old(t, add_special_tokens=False).input_ids
          if len(tt) == 0:
              tt = [tokenizer_old.unk_token_id]
          idx = self.tokenizer.convert_tokens_to_ids(t)
          self.model.model.shared.weight.data[idx] = self.model.model.shared.weight.data[tt].mean(0)


  def expand_vocab(self, text_file: str):
        spm_prefix = 'spm_amis_16k'
        self.__train_sentencepiece(text_file, spm_prefix)
        new_spm_model = self.__load_sentencepiece_model(f'{spm_prefix}.model')
        updated_spm_model = self.__update_existing_model_with_new_tokens(new_spm_model)
        new_spm_name = 'spm_nllb_amis_268k.model'
        self.__save_updated_model(updated_spm_model, new_spm_name)
        self.__update_tokenizer_and_model(new_spm_name)


  # Unfinished
  def evaluate(self):
    pd.Series(self.losses).ewm(100).mean().plot()


  def train(self, data: object, split: bool=False,
            epochs: int=25000, save_interval: int=5000,
            save_to: str="'/content/drive/MyDrive/MTApplication/models/nllb-eng-ami-v1'"):

    self.__setup()
    self.__generate_splits(data, split)
    self.__cleanup()
    self.model.cuda();
    self.model.train()

    for i in tqdm(range(epochs)):
      xx, yy, lang1, lang2 = self.__get_batch_pairs()
      try:
          # Tokenization and moving tensors to GPU
          self.tokenizer.src_lang = lang1
          x = self.__tokenize_and_transition(xx)
          y = self.__tokenize_and_transition(yy)
          self.tokenizer.tgt_lang = lang2

          y.input_ids[y.input_ids == self.tokenizer.pad_token_id] = -100

          # Forward and backward passes
          self.optimizer.zero_grad()
          loss = self.model(**x, labels=y.input_ids).loss
          loss.backward()
          self.optimizer.step()
          self.scheduler.step()

          self.losses.append(loss.item())
          if i % 500 == 0:
              print(f"Step {i}, Loss: {loss.item()}")

          # Checkpoint saving
          if i % save_interval == 0 and i > 0:
              current_loss_avg = np.mean(self.losses[-1000:])
              print(f"Average Loss last 1000 steps: {current_loss_avg}")
              self.model.save_pretrained(f'{self.MODEL_SAVE_PATH}/checkpoint_{i}')
              self.tokenizer.save_pretrained(f'{self.MODEL_SAVE_PATH}/checkpoint_{i}')

      except RuntimeError as e:
          self.optimizer.zero_grad()
          self.cleanup()
          print('RuntimeError:', e)
          continue

      except Exception as e:
          self.optimizer.zero_grad()
          print('An error occurred:', e)
          break

    self.model.save_pretrained(f'{save_to}/final')
    self.tokenizer.save_pretrained(f'{save_to}/final')